# Description requests

## Setup

In [ ]:
# Import modules
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [ ]:
# API connexion
client_id = 'cec8979c027344f98b471a991aa415ad'
client_secret = 'ad69bcba55b349f98c1344b006c708bd'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


## Get playlists description

In [ ]:
# Load data
df = pd.read_excel('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_requests/playlists_info_clean.xlsx')

# Remove playlists with "This is" and "Written By" in the name
df = df[~df['playlist_name'].str.contains('This Is', na=False)]
df = df[~df['playlist_name'].str.contains('Written By', na=False)]


In [ ]:
# API scraping
playlists_ids = df['playlist_id'].tolist()

playlists = []

for playlist_id in playlists_ids:
    print(f"Processing playlist: {playlist_id}")

    playlists_results = sp.playlist(playlist_id)

    playlist_name = playlists_results['name']
    playlist_description = playlists_results['description']

    playlists.append({
        "playlist_id": playlist_id, 
        "playlist_name": playlist_name,
        "playlist_description": playlist_description
    })

# Create dataframe from playlists
df_playlists = pd.DataFrame(playlists, columns=["playlist_id", "playlist_name", "playlist_description"])
df_playlists.to_excel("playlists_descriptions.xlsx", index=False)

print(df_playlists)


## Process playlists descrption with text analysis

## Merge with panel data